In [15]:
from pprint import pprint
import pickle
import numpy as np
import tensorflow as tf
import pandas as pd
import sys
import time
sys.path.append("../sketchformer")
sys.path.append("../src")


In [16]:
with open('../outputs/sample.pkl', 'rb') as f:
    sample = pickle.load(f)
print(len(sample), len(sample[0]), len(sample[0][0]['sketch']))
pprint(sample[0])


16 22 56
[{'label': 'tree',
  'position': [715, 390],
  'sketch': [[-4, 52, 0],
             [-30, 88, 0],
             [-28, 54, 1],
             [143, -175, 0],
             [-3, 5, 0],
             [-7, 71, 0],
             [0, 31, 0],
             [3, 15, 0],
             [12, 29, 0],
             [42, 46, 1],
             [-123, -206, 0],
             [-32, -1, 0],
             [-27, -5, 0],
             [-10, -3, 0],
             [-16, -11, 0],
             [-11, -23, 0],
             [0, -48, 0],
             [24, -29, 0],
             [10, -7, 0],
             [17, -5, 0],
             [7, 7, 0],
             [-1, 26, 0],
             [-5, 4, 0],
             [-5, -10, 0],
             [-2, -31, 0],
             [8, -20, 0],
             [15, -24, 0],
             [10, -9, 0],
             [14, -5, 0],
             [21, 1, 0],
             [5, 3, 0],
             [8, 16, 0],
             [0, 18, 0],
             [13, -29, 0],
             [7, -6, 0],
             [14, -9, 0],
 

In [17]:
from basic_usage.sketchformer import continuous_embeddings
sketchformer = continuous_embeddings.get_pretrained_model()


[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[run-experiment] resorting checkpoint if exists
[Checkpoint] Restored, step #207536


In [18]:
# class label
df = pd.read_csv('../outputs/sketchyscene_quickdraw.csv')
df = df.dropna(subset=['quickdraw_label'])
class_names = ['none']
for row in df.itertuples():
    class_names.append(row.quickdraw_label)
class_to_num = dict(zip(class_names, range(0, len(class_names))))

print(class_names)
print(len(class_names))
print(class_to_num)


['none', 'airplane', 'apple', 'hot air balloon', 'banana', 'basket', 'bee', 'bench', 'bicycle', 'bird', 'wine bottle', 'bucket', 'bus', 'butterfly', 'car', 'cat', 'chair', 'cloud', 'cow', 'cup', 'dog', 'duck', 'fence', 'flower', 'grapes', 'grass', 'horse', 'house', 'moon', 'mountain', 'face', 'pig', 'rabbit', 'sheep', 'star', 'streetlight', 'sun', 'table', 'tree', 'truck', 'umbrella']
41
{'none': 0, 'airplane': 1, 'apple': 2, 'hot air balloon': 3, 'banana': 4, 'basket': 5, 'bee': 6, 'bench': 7, 'bicycle': 8, 'bird': 9, 'wine bottle': 10, 'bucket': 11, 'bus': 12, 'butterfly': 13, 'car': 14, 'cat': 15, 'chair': 16, 'cloud': 17, 'cow': 18, 'cup': 19, 'dog': 20, 'duck': 21, 'fence': 22, 'flower': 23, 'grapes': 24, 'grass': 25, 'horse': 26, 'house': 27, 'moon': 28, 'mountain': 29, 'face': 30, 'pig': 31, 'rabbit': 32, 'sheep': 33, 'star': 34, 'streetlight': 35, 'sun': 36, 'table': 37, 'tree': 38, 'truck': 39, 'umbrella': 40}


In [19]:
def convert_dataset_to_xy(dataset):
    input_batch = []
    label_batch = []
    for scene in dataset:
        sketches = list(map(lambda o: o['sketch'], scene))
        sketch_embeddings = sketchformer.get_embeddings(sketches)
        input_scene = []
        labels = []
        for se, obj in zip(sketch_embeddings, scene):
            p = [obj['position'][0] / 750, obj['position'][1] / 750]
            o = se.numpy().tolist() + p
            input_scene.append(o)  # オブジェクトの数が不規則
            labels.append(class_to_num[obj['label']])  # convert to num
        input_batch.append(input_scene)
        label_batch.append(labels)
    return tf.ragged.constant(input_batch).to_tensor(0.), tf.ragged.constant(label_batch).to_tensor(0)


In [20]:
x, y = convert_dataset_to_xy(sample)
print(x.shape)
print(y.shape)
print(x)
print(y)


[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (22, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (9, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (11, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (10, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (7, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extra

In [21]:
# Create masks

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # アテンション・ロジットにパディングを追加するため
    # さらに次元を追加する
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def create_masks(inp, tar):
    # Encoderパディング・マスク
    enc_padding_mask = create_padding_mask(inp)

    # デコーダーの 2つ目のアテンション・ブロックで使用
    # このパディング・マスクはエンコーダーの出力をマスクするのに使用
    dec_padding_mask = create_padding_mask(inp)

    # デコーダーの 1つ目のアテンション・ブロックで使用
    # デコーダーが受け取った入力のパディングと将来のトークンをマスクするのに使用
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


def create_combined_mask(tar):

    # デコーダーの 1つ目のアテンション・ブロックで使用
    # デコーダーが受け取った入力のパディングと将来のトークンをマスクするのに使用
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return combined_mask


In [22]:
from isketcher import InteractiveSketcher

interactive_sketcher_test = InteractiveSketcher(
    num_layers=2, d_model=130, num_heads=5, dff=2048,
    object_num=40, pe_target=40)

combined_mask = create_combined_mask(y)

c_out, x_out, y_out, _ = interactive_sketcher_test(
    x, training=False, look_ahead_mask=combined_mask)

print(c_out.shape, x_out.shape, y_out.shape)
print(c_out)  # class (one-hot)
print(x_out)  # x (0 ~ 1)
print(y_out)  # y (0 ~ 1)


(16, 26, 40) (16, 26) (16, 26)
tf.Tensor(
[[[0.01166196 0.02416692 0.08382018 ... 0.00204804 0.06846843 0.01784139]
  [0.01076908 0.01562492 0.06401125 ... 0.00211613 0.04532384 0.04048114]
  [0.01583777 0.0164714  0.03268166 ... 0.00355447 0.07555916 0.03261128]
  ...
  [0.00427682 0.05154302 0.10233366 ... 0.01388562 0.00896969 0.00550187]
  [0.00467705 0.05079884 0.09835845 ... 0.01364191 0.00815326 0.00491598]
  [0.00135909 0.06744608 0.11819236 ... 0.0106847  0.00748545 0.00724206]]

 [[0.00406517 0.01105853 0.01629733 ... 0.00984348 0.00150961 0.01246477]
  [0.01283476 0.0092246  0.07543013 ... 0.00651636 0.02255101 0.01228609]
  [0.0018785  0.00802625 0.01391925 ... 0.00794254 0.01293164 0.04172643]
  ...
  [0.00314379 0.00951818 0.01551305 ... 0.00123968 0.01431587 0.00562057]
  [0.00332654 0.00988419 0.01388834 ... 0.00153632 0.01843716 0.00727322]
  [0.001746   0.00568538 0.01253563 ... 0.00254721 0.01689883 0.00457083]]

 [[0.00198458 0.02424331 0.02159152 ... 0.03480235 0.0

In [23]:
# Hyper parameters

num_layers = 4
d_model = 130
dff = 512
num_heads = 5

target_object_num = 40  # object num
dropout_rate = 0.1


In [24]:
# Optimizer

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)


In [25]:
interactive_sketcher = InteractiveSketcher(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
    object_num=target_object_num, pe_target=target_object_num, rate=dropout_rate)


In [26]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=interactive_sketcher,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# チェックポイントが存在したなら、最後のチェックポイントを復元
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')


Latest checkpoint restored!!


In [27]:
# Training step

# @tf.functionは高速に実行するためにtrain_stepをTFグラフにトレースコンパイルします。
# この関数は、引数となるテンソルのshapeに特化したものです。
# シーケンスの長さや（最後のバッチが小さくなるなど）バッチサイズが可変となることによって
# 再トレーシングが起きないようにするため、input_signatureを使って、より一般的なshapeを
# 指定します。

train_step_signature = [
    tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),
    tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),
]

scc = tf.keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE)
mse = tf.keras.losses.MeanSquaredError(
    reduction=tf.keras.losses.Reduction.NONE)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_class_accuracy')


# @tf.function(input_signature=train_step_signature) # なぜか通らないため一旦コメントアウト
def train_step(tar, labels):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    labels_inp = labels[:, :-1]
    labels_real = labels[:, 1:]

    # パディングしたオブジェクトの位置はlabelsが0の位置のため、そこからマスクを作成
    combined_mask = create_combined_mask(labels_inp)

    with tf.GradientTape() as tape:
        c_out, x_out, y_out, _ = interactive_sketcher(
            tar_inp, True, combined_mask)

        # class loss
        # クラスラベルはカテゴリカルクロスエントロピー
        c_loss_ = scc(labels_real, c_out)

        # position loss
        # 位置座標は平均二乗誤差
        x_real, y_real = tar_real[:, :, -2], tar_real[:, :, -1]
        p_loss_ = tf.math.square(x_real - x_out) + \
            tf.math.square(y_real - y_out)

        # mask padded object
        # パディングしたオブジェクトの部分を損失に加えないようにマスクする
        mask = tf.math.logical_not(tf.math.equal(labels_real, 0))
        c_mask = tf.cast(mask, dtype=c_loss_.dtype)
        c_loss = tf.reduce_mean(c_loss_ * c_mask)
        p_mask = tf.cast(mask, dtype=p_loss_.dtype)
        p_loss = tf.reduce_mean(p_loss_ * p_mask)

        loss = c_loss + p_loss

    gradients = tape.gradient(loss, interactive_sketcher.trainable_variables)
    optimizer.apply_gradients(
        zip(gradients, interactive_sketcher.trainable_variables))

    train_loss(loss)
    train_accuracy(labels_real, c_out)


In [28]:
# Training

EPOCHS = 0

for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    # for (batch, (inp, tar)) in enumerate(train_dataset):
    #   train_step(inp, tar)

    #   if batch % 50 == 0:
    #     print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
    #         epoch + 1, batch, train_loss.result(), train_accuracy.result()))

    # とりあえず1epoch-1batchでやってみる
    train_step(x, y)
    print('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(
        epoch + 1, train_loss.result(), train_accuracy.result()))

    if (epoch + 1) % 1000 == 0:
        ckpt_save_path = ckpt_manager.save()
        print('Saving checkpoint for epoch {} at {}'.format(epoch + 1,
                                                            ckpt_save_path))

    print('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                        train_loss.result(),
                                                        train_accuracy.result()))

    print('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


In [31]:
# validation

scene_index = 0

print("----", 0, "-------")
print("real label: ", sample[scene_index][0]["label"])
print("real position: ", sample[scene_index][0]["position"])

for i in range(1, len(sample[scene_index])):

    c_out, x_out, y_out, _ = interactive_sketcher(
        x[:, :i], training=False, look_ahead_mask=None)

    print("----", i, "-------")
    print("real label: ", sample[scene_index][i]["label"])
    print("pred label: ", class_names[tf.argmax(c_out[scene_index][0])])

    print("real position: ", sample[scene_index][i]["position"])
    print("pred position:  [{0}, {1}]".format(round(x_out[scene_index][0].numpy() * 750, 2), round(y_out[scene_index][0].numpy() * 750, 2)))


---- 0 -------
real label:  tree
real position:  [715, 390]
---- 1 -------
real label:  tree
pred label:  tree
real position:  [690, 320]
pred position: [705.8, 342.78]
---- 2 -------
real label:  tree
pred label:  tree
real position:  [610, 290]
pred position: [678.75, 324.43]
---- 3 -------
real label:  cloud
pred label:  cloud
real position:  [225, 90]
pred position: [634.6, 191.14]
---- 4 -------
real label:  tree
pred label:  cloud
real position:  [340, 355]
pred position: [679.37, 126.48]
---- 5 -------
real label:  cloud
pred label:  cloud
real position:  [665, 105]
pred position: [682.24, 125.75]
---- 6 -------
real label:  tree
pred label:  tree
real position:  [50, 365]
pred position: [501.36, 248.02]
---- 7 -------
real label:  tree
pred label:  tree
real position:  [105, 225]
pred position: [231.07, 409.85]
---- 8 -------
real label:  tree
pred label:  tree
real position:  [220, 260]
pred position: [246.71, 396.32]
---- 9 -------
real label:  tree
pred label:  tree
real pos